In [26]:
import math
import numpy as np
import matplotlib.pyplot as pp
% matplotlib inline

import libstempo

In [117]:
# we'll use this throughout our tests
psr = libstempo.tempopulsar('../2222.par','../2222.tim')

In [226]:
# I didn't have time to study your parameter implementation, so this is my own

class Parameter(object):
    def __init__(self,name):
        self.name = name

    # this trick lets us pass an instantiated parameter to a signal;
    # the parameter will refuse to be renamed and will return itself
    def __call__(self,name):
        return self
        
# the following two are class *factories*: they return a customized class
# then the actual parameters (with names) are obtained by instantiating the class

def Uniform(pmin,pmax):
    # this naive class definition can be improved with type, etc.
    # but will be good enough for a demo
    class Uniform(Parameter):
        def pdf(self,p):
            return 1.0 / (pmax - pmin) if (pmin < p < pmax) else 0.0
                
        def __repr__(self):
            return '"{}":Uniform({},{})'.format(self.name,pmin,pmax)
        
    return Uniform

def Normal(mu=0,sigma=1):
    class Normal(Parameter):
        norm = 1.0 / math.sqrt(2 * math.pi * sigma**2)
        
        def pdf(self,x):
            return self.norm * math.exp(-0.5*(x - mu)**2/sigma**2)
        
    return Normal

In [227]:
# a uniformly distributed parameter
u = Uniform(0,10)
u

__main__.Uniform.<locals>.Uniform

In [228]:
# instantiations of this parameter
u1 = u('uniform1')
u2 = u('uniform2')

In [229]:
u1

"uniform1":Uniform(0,10)

In [230]:
u1.pdf(5), u1.pdf(-1)

(0.1, 0.0)

In [231]:
n = Normal(2,5)

In [232]:
n1 = n('normal1')
n2 = n('normal2')

In [233]:
n1.pdf(2)

0.07978845608028654

In [201]:
# base signal class. Not much in it yet
class Signal(object):
    @property
    def params(self):
        return self._params.values()
    
    def ndiag(self,params):
        return None
    
    def Fmat(self,params):
        return None
    
    def Phivec(self,params):
        return None

In [236]:
# make a collection of signals from a list of signal classes
# (this is a factor; the scheme would work also by subclassing an existing
# SignalCollection class that defines all these methods...)

def SignalCollection(metasignals):
    def __init__(self,psr):
        # instantiate all the signals with a pulsar
        self._signals = [metasignal(psr) for metasignal in self._metasignals]
    
    # we may consider defining __add__ also for instantiated signals. Why not... 
    
    @property
    def params(self):
        # this would allow duplicates
        # return set(param for signal in self._signals for param in signal.params)

        # no duplicates, but expensive, so a candidate for memoization
        ret = []
        for signal in self._signals:
            for param in signal.params:
                if param not in ret:
                    ret.append(param)
        
        return ret
                    
    # there may be a smarter way to write these...
    
    def ndiag(self,params):
        ndiags = [signal.ndiag(params) for signal in self._signals]
        return sum(ndiag for ndiag in ndiags if ndiag is not None)
    
    def Fmat(self,params):
        Fmats = [signal.Fmat(params) for signal in self._signals]
        return np.hstack(Fmat for Fmat in Fmats if Fmat is not None)
    
    def Phivec(self,params):
        Phivecs = [signal.Phivec(params) for signal in self._signals]
        return np.hstack(Phivec for Phivec in Phivecs if Phivec is not None)
    
    return MetaCollection('SignalCollection',(),
                          {'__init__': __init__,
                           'params': params,
                           'ndiag': ndiag,
                           'Fmat': Fmat,
                           'Phivec': Phivec,
                           '_metasignals': metasignals})

In [237]:
# since we wish to add Signal classes (not instances) together,
# we need to make them with a custom metaclass that defines __add__

class MetaSignal(type):
    def __add__(self,other):
        if isinstance(other,MetaSignal):
            return SignalCollection([self,other])
        elif isinstance(other,MetaCollection):
            return SignalCollection([self] + other._metasignals)
        else:
            raise TypeError
    
class MetaCollection(type):
    def __add__(self,other):
        if isinstance(other,MetaSignal):
            return SignalCollection(self._metasignals + [other])
        elif isinstance(other,MetaCollection):
            return SignalCollection(self._metasignals + other._metasignals)
        else:
            raise TypeError

In [238]:
# again, a factory that returns a customized class, reusable to generate
# signal instances for many pulsars

def MeasurementNoise(efac = Uniform(0.5,1.5)):
    def __init__(self,psr):
        self._psr = psr

        self._params = {'efac': efac(psr.name + '_efac')}
        self._ndiag = (psr.toaerrs*1e-6)**2

    # here we assume that we pass all parameters to all the "tensor"
    # methods of the instantiated signals. An alternative is to set
    # a persistent value for each parameter. It's less functional,
    # but (often) practicality beats purity
    def ndiag(self,params):
        return params[self._params['efac'].name]**2 * self._ndiag

    return MetaSignal('MeasurementNoise',(Signal,),
                      {'__init__': __init__,
                       'ndiag': ndiag})

In [203]:
def EquadNoise(log10_equad = Uniform(-18,-14)):
    def __init__(self,psr):
        self._psr = psr

        self._params = {'log10_equad': log10_equad(psr.name + '_log10_equad')}
        self._ndiag = np.ones_like(psr.toaerrs)

    def ndiag(self,params):
        return 10.0**(2*params[self._params['log10_equad'].name]) * self._ndiag

    return MetaSignal('EquadNoise',(Signal,),
                      {'__init__': __init__,
                       'ndiag': ndiag})

In [239]:
w = MeasurementNoise()

In [240]:
w1 = w(psr)

In [241]:
w1.params

dict_values(["J2222-0137_efac":Uniform(0.5,1.5)])

In [242]:
w1.ndiag({'J2222-0137_efac': 1.0})[0]

3.7423676303999994e-10

In [243]:
w1.ndiag({'J2222-0137_efac': 1.5})[0]

8.4203271683999982e-10

In [244]:
x = MeasurementNoise(efac=Normal(1,0.5))

In [245]:
x1 = x(psr)

In [246]:
x1.params

dict_values([<__main__.Normal.<locals>.Normal object at 0x10df996d8>])

In [247]:
e = EquadNoise()

In [248]:
# combining two signals!
m = w + e

In [249]:
m1 = m(psr)

In [250]:
m1._signals

In [251]:
m1.params

["J2222-0137_efac":Uniform(0.5,1.5), "J2222-0137_log10_equad":Uniform(-18,-14)]

In [252]:
# I think the relative units are wrong, by the way.
m1.ndiag({'J2222-0137_efac': 1.0,'J2222-0137_log10_equad': -16})[:20]

array([  3.74236763e-10,   4.02764761e-10,   4.40588496e-10,
         5.52272090e-10,   5.50109348e-10,   1.02201249e-09,
         5.37069501e-10,   7.27812484e-10,   9.21936667e-10,
         9.30346993e-10,   3.05499398e-11,   2.33897977e-11,
         3.46814988e-11,   1.28601132e-11,   1.21619588e-11,
         1.17073466e-11,   8.18572562e-11,   5.47970062e-11,
         6.71105024e-11,   1.77342054e-11])

In [253]:
def Function(f,**kwargs):
    class Function(object):
        def __init__(self,prefix):
            self._params = {kw: arg(prefix + '_' + kw) for kw,arg in kwargs.items()}
        
        # params could also be a standard argument here,
        # but by defining it as ** we allow multiple positional arguments
        def __call__(self,*args,**params):
            pardict = {kw: params[par.name] for kw,par in self._params.items()
                                            if par.name in params}
            return f(*args,**pardict)
        
        @property
        def params(self):
            return self._params.values()
    
    return Function

In [255]:
year = 365.25 * 24 * 3600

# unfortunately lambdas are not acceptable here because of the way we call them
# a def is not too bad though; all sampling parameters must go in as keywords
def powerlaw(f,log10_A=-16,gamma=5):
    return (10**log10_A) * (f*year)**(-gamma)

PowerLaw = Function(powerlaw,log10_A=Uniform(-18,-12),gamma=Uniform(1,7))

In [256]:
PowerLaw

__main__.Function.<locals>.Function

In [257]:
pw = PowerLaw('J2222-0137')

In [258]:
pw.params

dict_values(["J2222-0137_log10_A":Uniform(-18,-12), "J2222-0137_gamma":Uniform(1,7)])

In [259]:
pw._params

{'gamma': "J2222-0137_gamma":Uniform(1,7),
 'log10_A': "J2222-0137_log10_A":Uniform(-18,-12)}

In [260]:
pw(1e-7,**{'J2222-0137_gamma': 5,"J2222-0137_log10_A": -16})

3.1950684338930047e-19

In [263]:
# defaults are respected
pw(1e-7,**{"J2222-0137_log10_A": -16})

3.1950684338930047e-19

In [264]:
# start with marginalized only... my actual matrices may be wrong, since I did them very fast

def FourierBasisGP(spectrum=None,components=20):
    def __init__(self,psr):
        self._psr = psr

        self._spectrum = spectrum(psr.name)
        self._params = self._spectrum._params

        self._toas = psr.stoas

        # should use a common epoch? no subtraction at all?
        self._t = 86400.0 * (self._toas - np.min(self._toas))
        self._T = np.max(self._toas) - np.min(self._toas)

        self._f = np.arange(1,components+1) / self._T

        self._f2 = np.zeros(2*len(self._f),'d')
        self._f2[0::2] = self._f2[1::2] = self._f

        self._F = np.zeros((len(self._t),2*len(self._f)),'d')
        for i in range(components):
            self._F[:,2*i]   = np.cos(2*math.pi*self._f[i]*self._t)
            self._F[:,2*i+1] = np.sin(2*math.pi*self._f[i]*self._t)

    def Fmat(self,params=None):
        return self._F

    # maybe we only need diagonal Phi?
    def Phivec(self,params):
        return self._spectrum(self._f2,**params)

    return MetaSignal('FourierBasisGP',(Signal,),
                      {'__init__': __init__,
                       'Fmat': Fmat,
                       'Phivec': Phivec})

In [265]:
f = FourierBasisGP(spectrum=PowerLaw)

In [266]:
f1 = f(psr)

In [267]:
f1.params

dict_values(["J2222-0137_log10_A":Uniform(-18,-12), "J2222-0137_gamma":Uniform(1,7)])

In [268]:
f1._f

array([ 0.0010900472,  0.0021800945,  0.0032701417,  0.004360189,
        0.0054502362,  0.0065402835,  0.0076303307,  0.008720378,
        0.0098104252,  0.010900472,  0.01199052,  0.013080567,
        0.014170614,  0.015260661,  0.016350709,  0.017440756,
        0.018530803,  0.01962085,  0.020710898,  0.021800945], dtype=float128)

In [269]:
fphi = f1.Phivec({"J2222-0137_gamma": 5, "J2222-0137_log10_A": -16})

In [284]:
n = m + f

In [285]:
n1 = n(psr)

In [286]:
n1.params

["J2222-0137_efac":Uniform(0.5,1.5),
 "J2222-0137_log10_equad":Uniform(-18,-14),
 "J2222-0137_log10_A":Uniform(-18,-12),
 "J2222-0137_gamma":Uniform(1,7)]

In [287]:
n1.ndiag({'J2222-0137_efac': 1.0,'J2222-0137_log10_equad': -16,
         'J2222-0137_gamma': 5, 'J2222-0137_log10_A': -16}).shape

(244,)

In [288]:
n1.Phivec({'J2222-0137_efac': 1.0,'J2222-0137_log10_equad': -16,
           'J2222-0137_gamma': 5, 'J2222-0137_log10_A': -16}).shape

(40,)

In [289]:
n1.Fmat({'J2222-0137_efac': 1.0,'J2222-0137_log10_equad': -16,
         'J2222-0137_gamma': 5, 'J2222-0137_log10_A': -16}).shape

(244, 40)